In [ ]:
import pandas as pd
from datetime import datetime, date, timedelta
import requests

In [ ]:
list_for_df = []
def tomorrows_flight_arrivals(icao_list):
  tomorrow = date.today()+timedelta(1)
  for icao in icao_list:
    times = [["00:00","11:59"],["12:00","23:59"]]

    for time in times:

      url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{icao}/{tomorrow}T{time[0]}/{tomorrow}T{time[1]}"
      querystring = {"withLeg":"true","direction":"Arrival","withCancelled":"false","withCodeshared":"true","withCargo":"false","withPrivate":"false"}
      headers = {'x-rapidapi-host': "aerodatabox.p.rapidapi.com", 'x-rapidapi-key': "dca4cdbc0cmsheed63a6cfd5f6e5p141deejsnfcab75bd2e58" }
      response = requests.request("GET", url, headers=headers, params=querystring)
      flights_json = response.json()

      for flight in flights_json['arrivals']:
        flights_dict = {}
        flights_dict['arrival_icao'] = icao
        flights_dict['arrival_time_local'] = flight['arrival'].get('scheduledTimeLocal', None)
        flights_dict['arrival_terminal'] = flight['arrival'].get('terminal', None)
        flights_dict['departure_city'] = flight['departure']['airport'].get('name', None)
        flights_dict['departure_icao'] = flight['departure']['airport'].get('icao', None)
        flights_dict['departure_time_local'] = flight['departure'].get('scheduledTimeLocal', None)
        flights_dict['airline'] = flight['airline'].get('name', None)
        flights_dict['flight_number'] = flight.get('number', None)
        flights_dict['data_retrieved_on'] = date.today()
        list_for_df.append(flights_dict)

  return pd.DataFrame(list_for_df)

In [ ]:
icaos = ['EDDB','EDDH','EGLL','EGCC','LEBL']
flight_data = tomorrows_flight_arrivals(icaos)
import numpy as np
condition=[(flight_data['arrival_icao'] == 'EDDB') , (flight_data['arrival_icao'] == 'EDDH') ,
           (flight_data['arrival_icao'] == 'EGLL') , (flight_data['arrival_icao'] == 'EGCC'),
           (flight_data['arrival_icao'] == 'LEBL')]
choice=['berlin','hamburg','london','manchester','barcelona']
flight_data['city_name'] =np.select(condition,choice,default = '')



In [ ]:
import numpy as np
flight_data['flight_id'] = np.arange(1, len(flight_data) + 1)
flight_data=flight_data.iloc[:,[10,7,0,1,2,3,4,5,6,8,9]]
flight_data

,data_retrieved_on,departure_city,city_name,departure_time_local,flight_id,flight_number,arrival_icao,arrival_time_local,arrival_terminal,departure_icao,airline
0,2023-03-18,Doha,berlin,2023-03-19 02:25+03:00,1,QR 79,EDDB,2023-03-19 06:50+01:00,1,OTHH,Qatar Airways
1,2023-03-18,Frankfurt-am-Main,berlin,2023-03-19 06:45+01:00,2,LH 170,EDDB,2023-03-19 07:55+01:00,1,EDDF,Lufthansa
2,2023-03-18,New York,berlin,None,3,DL 92,EDDB,2023-03-19 07:05+01:00,1,None,Delta Air Lines
3,2023-03-18,Bâle/Mulhouse,berlin,2023-03-19 06:20+01:00,4,U2 5632,EDDB,2023-03-19 07:55+01:00,1,LFSB,easyJet
4,2023-03-18,Newark,berlin,2023-03-18 18:50-04:00,5,UA 962,EDDB,2023-03-19 07:55+01:00,1,KEWR,United
...,...,...,...,...,...,...,...,...,...,...,...
17623,2023-03-18,Menorca Island,barcelona,2023-03-19 21:30+01:00,17624,VY 3719,LEBL,2023-03-19 22:20+01:00,1,LEMH,Vueling
17624,2023-03-18,Palma De Mallorca,barcelona,2023-03-19 21:55+01:00,17625,VY 3917,LEBL,2023-03-19 22:50+01:00,1,LEPA,Vueling
17625,2023-03-18,Palma De Mallorca,barcelona,2023-03-19 21:05+01:00,17626,VY 3921,LEBL,2023-03-19 22:00+01:00,1,LEPA,Vueling
17626,2023-03-18,Zurich,barcelona,2023-03-19 20:35+01:00,17627,VY 6249,LEBL,2023-03-19 22:15+01:00,1,LSZH,Vueling


In [ ]:
!pip install sqlalchemy
!pip install pymysql
!pip install mysql-connector-python-rf
import sqlalchemy # install if needed
import pymysql
from sqlalchemy import create_engine
import mysql.connector
import socket

schema="gans"   # name of the database you want to use here
host="localhost"        # to connect to your local server
user="root"
password="Payment_123" # your password!!!!
port=3306
con = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

socket.getaddrinfo('localhost', 8080)
cnx = create_engine(con,echo=False)
df = pd.read_sql('SELECT * FROM flight_data', con)

flight_data.to_sql('flight_data', con=cnx, if_exists='append', index=False)

17628